In [18]:
!py -m pip install nltk


# !py -m pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai langchain-chroma bs4


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
!py -m pip list

Package                                  Version
---------------------------------------- -------------------
absl-py                                  1.0.0
aiohttp                                  3.9.5
aiosignal                                1.3.1
altair                                   5.3.0
annotated-types                          0.6.0
anyio                                    3.5.0
argon2-cffi                              21.3.0
argon2-cffi-bindings                     21.2.0
asgiref                                  3.8.1
asttokens                                2.0.5
astunparse                               1.6.3
async-timeout                            4.0.3
asyncio                                  3.4.3
attrs                                    21.4.0
Babel                                    2.9.1
backcall                                 0.2.0
backoff                                  2.2.1
bcrypt                                   4.1.3
beautifulsoup4                           

opentelemetry-sdk                        1.24.0
opentelemetry-semantic-conventions       0.45b0
opentelemetry-util-http                  0.45b0
opt-einsum                               3.3.0
ordered-set                              4.1.0
orjson                                   3.10.1
overrides                                7.7.0
packaging                                23.2
pandas                                   2.2.3
pandocfilters                            1.5.0
parso                                    0.8.3
pathspec                                 0.9.0
patsy                                    0.5.6
pdf2image                                1.17.0
pickleshare                              0.7.5
pillow                                   10.4.0
pip                                      24.3.1
pipreqs                                  0.5.0
platformdirs                             4.3.6
playsound                                1.3.0
plotly                                   5.19.0
plotni

In [20]:
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
import pandas as pd
import os
import shutil
import numpy as np
from uuid import uuid4
from langchain_community.document_loaders import PyPDFLoader, UnstructuredWordDocumentLoader
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
import faiss
from langchain.prompts import ChatPromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.docstore.document import Document
from typing import List, Tuple
from langchain.retrievers import ParentDocumentRetriever
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain.storage import InMemoryStore
import re

In [2]:
OPENAI_DEPLOYMENT_ENDPOINT = "https://pkl-aa-dev-aiservices.openai.azure.com/" 
OPENAI_API_KEY = "AXEC3y1jC9ZNGCBB12NZwrpBSzScq1esexgvCXiqw7PaHE04vSMbJQQJ99BDACYeBjFXJ3w3AAABACOG4CMN" 
OPENAI_API_VERSION = "2024-12-01-preview"

OPENAI_DEPLOYMENT_NAME = "gpt-4o"
OPENAI_MODEL_NAME="gpt-4o"
embedding_api_version = "2024-02-01"

# OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
# CHROMA_PATH = "c:\\Users\\SujaySunilNagvekar\\VM\\GEN AI\\SERFF\\vector_db\\testing_db"
DB_PATH  = "C:\\Users\\SujaySunilNagvekar\\VM\\GEN AI\\KM\\vector_db"
DATA_FOLDER = "C:\\Users\\SujaySunilNagvekar\\VM\\GEN AI\\KM\\Documents"

### Cohere API key 
# cohere_API_key = "wHyiTViP32Y3Q8Qwhjmd4QGNCkYNpxqtsemtSri3"
# co = cohere.Client(cohere_API_key)

In [3]:

llm = AzureChatOpenAI(
                        temperature=0,
                        deployment_name=OPENAI_DEPLOYMENT_NAME,
                        model_name=OPENAI_MODEL_NAME,
                        azure_endpoint=OPENAI_DEPLOYMENT_ENDPOINT,
                        openai_api_version=OPENAI_API_VERSION,
                        openai_api_key=OPENAI_API_KEY            
                    )

embeddings = AzureOpenAIEmbeddings(
                        deployment="text-embedding-3-small",
                        model="text-embedding-3-small",
                        azure_endpoint=OPENAI_DEPLOYMENT_ENDPOINT,
                        openai_api_version=embedding_api_version,
                        openai_api_key=OPENAI_API_KEY)


### Chunking and Embedding

In [42]:
# 🔧 Config

# Splitters
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=100)


# Create empty FAISS index
dim = len(embeddings.embed_query("test"))  # Get embedding size
faiss_index = faiss.IndexFlatL2(dim)


# Create embedding-based FAISS vector index for child chunks
vectorstore = FAISS(
    embedding_function=embeddings,      # your embedding model
    index=faiss_index,                  # faiss.IndexFlatL2(dim)
    docstore=InMemoryDocstore(),        # stores child chunks
    index_to_docstore_id={}             # maps FAISS ids to document IDs
)

# Create parent document store for full/large chunks
docstore = InMemoryStore()
    
# Parent-Child retriever
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=docstore,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter
)

In [43]:
# Load docs
docs = []

for filename in os.listdir(DATA_FOLDER):
    file_path = os.path.join(DATA_FOLDER, filename)

    if filename.endswith(".pdf"):
        loader = PyPDFLoader(file_path)
    elif filename.endswith(".docx"):
        loader = UnstructuredWordDocumentLoader(file_path)
    else:
        continue

    pages = loader.load()
    full_text = "\n\n".join([page.page_content for page in pages])
    doc = Document(page_content=full_text, metadata={"filename": filename})
    docs.append(doc)

retriever.add_documents(docs)  # ✅ THIS is critical

print("✅ Added to parent-child retriever!")

✅ Added to parent-child retriever!


In [44]:
vectorstore.save_local(DB_PATH)

import pickle

with open("parent_docstore.pkl", "wb") as f:
    pickle.dump(retriever.docstore.store, f)

### Retriever

In [45]:
# 1. Reload FAISS vectorstore (child chunks)
vectorstore = FAISS.load_local(
    DB_PATH,
    embeddings=embeddings,
    allow_dangerous_deserialization=True
)

# 2. Reload parent docstore
with open("parent_docstore.pkl", "rb") as f:
    parent_data = pickle.load(f)

docstore = InMemoryStore()
docstore.store = parent_data

# 3. Rebuild retriever
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=docstore,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

In [47]:
results = retriever.invoke("Team Concept")
for doc in results:
    print("📄 Filename:", doc.metadata.get("filename"))
    print("🧩 Chunk Size:", doc.metadata.get("chunk_size"))
    print("📏 Length:", len(doc.page_content))
    print(doc.page_content[:300], "\n---\n")


📄 Filename: Arch-Lit Guidelines-2014.pdf
🧩 Chunk Size: None
📏 Length: 1971
2                                                           LITIGATION GUIDELINES FOR DEFENSE AND COVERAGE COUNSEL 11/1/14  Team Concept  
The team concept is best implemented if the cases assigned b y Arch Insur ance are con sistently 
handled by a core group of legal professionals in your office.  
---

📄 Filename: Leader SW_Final_1.30.18.docx
🧩 Chunk Size: None
📏 Length: 1991
Auto Investigation Supervisor Frequency Task Key Outputs Weekly Workload Management Utilize reporting to evaluate employee productivity and compliance with established workload expectations (based on KPIs) Daily Case Reviews Review and provide feedback on all files at specific points in the claim li 
---

📄 Filename: Leader SW_Final_1.30.18.docx
🧩 Chunk Size: None
📏 Length: 920
summary of prior reporting period KPIs and results Share performance results with applicable parties Share individual performance during 1:1s Monthly 1:1 with Dir

In [6]:
PROMPT_TEMPLATE = """
You are an expert in interpreting SERFF (System for Electronic Rates & Forms Filing) documents. 
Your task is to provide clear, concise, and accurate answers based on the retrieved document context. 
The context includes both **parent** and **child** chunks of the relevant documents.

### **Instructions:**
- Provide answers based strictly on the provided context.
- Do not include irrelevant information.
- Avoid any form of hallucination or speculation.
- Present answers in bullet points for clarity and conciseness.
- If the answer is contained in a table, preserve the table structure and format.
- The Answer should be detailed and expansive
- For questions needing specific answers from a table, take as much time as needed to answer accurately. Pay special attention to numbers and counties in the question

---

### **User Query:**
{query}

---

### **Document Context:**

**Parent Chunks (Broader Context):**
{parent_context}

**Child Chunks (Specific Details):**
{child_context}

**Historical Context (Chat History Summary):**
{history_summary}
---

### **Your Answer:**
"""

# Initialize the prompt with ChatPromptTemplate
serff_prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)

In [7]:
# Initialize a list to store the last 5 question-answer pairs
conversation_history = []
history_limit = 5  # Keep up to the last 5 exchanges

def update_conversation_history(question, answer):
    """
    Store the last 5 interactions (Standalone Query + LLM Response).
    """
    conversation_history.append(f"Question: {question}\nAnswer: {answer}")
    if len(conversation_history) > history_limit:
        conversation_history.pop(0)  # Keep only the latest 5 entries

def clear_history():
    """
    Clear the entire conversation history.
    """
    global conversation_history
    conversation_history = []
    print("🧹 Conversation history cleared!")

def get_conversation_context():
    """
    Return the concatenated conversation history.
    """
    return "\n\n".join(conversation_history)


In [8]:
def retrieve_and_answer_the_query(query, llm, retriever, k=5):
    """
    Retrieve relevant chunks using hybrid retrieval. Reformulate and summarize only if chat history exists.
    """
    print(f"🔍 Searching for: {query}")

    # Step 1: Check if there's existing conversation history
    conversation_context = get_conversation_context()

    if conversation_context:
        # ✅ Summarize the previous Q&A to add as context
        summary_prompt = f"""
        You are an expert in understanding SERFF document conversations.
        Given the following conversation history, provide a concise summary that captures key points. Extract Topic , Entities , Theme ( one liner ) if any, to the best of your ability.

        Conversation History:
        {conversation_context}

        Enities:
        Theme:
        Summary:
        """
        start_time = time.time()
        summary_response = llm.invoke(summary_prompt, max_tokens=MAX_HISTORY_SUMMARY_TOKENS)
        summary_duration = time.time() - start_time        
        history_summary = summary_response.content.strip()
        summary_tokens = count_tokens(summary_prompt) + count_tokens(summary_response.content)
        print(f"📝 History Summary Generated in {summary_duration:.2f} seconds | Tokens Used: {summary_tokens}")

        # ✅ Reformulate the new query based on chat history
        reformulation_prompt = f"""
        Given the conversation history and the user's latest question, reformulate the user query as a standalone query.

        Conversation History:
        {conversation_context}

        Latest Question:
        {query}

        Reformulated Question:
        """
        start_time = time.time()
        reformulated_response = llm.invoke(reformulation_prompt)
        reformulation_duration = time.time() - start_time
        standalone_query = reformulated_response.content.strip()
        reformulation_tokens = count_tokens(reformulation_prompt) + count_tokens(reformulated_response.content)
        print(f"🔄 Reformulated Query: {standalone_query} | Time: {reformulation_duration:.2f}s | Tokens: {reformulation_tokens}")
    else:
        # No history: Use the original query directly
        standalone_query = query
        history_summary = ""  # No summary for the first question

    # Step 2: Retrieve Top-K Child Chunks using Hybrid Retrieval
    print(standalone_query)
    start_time = time.time()
    retrieved_children = retriever.invoke(standalone_query)
    retrieval_duration = time.time() - start_time
    print(f"✅ Retrieved {len(retrieved_children)} Child Chunks in {retrieval_duration:.2f} seconds")
    
    # Extract unique parent IDs from retrieved child chunks
    parent_ids = list(set([doc.metadata["related_to"] for doc in retrieved_children]))    
    print(f"📥 Fetching Parent Chunks for Parent_IDs: {parent_ids}")

    # Step 3: Retrieve Corresponding Parent Chunks based on Parent IDs
    start_time = time.time()
    parent_results = parent_db.similarity_search(standalone_query, k=5, filter={"ID": {"$in": parent_ids}})
    parent_retrieval_duration = time.time() - start_time
    print(f"✅ Retrieved {len(parent_results)} Parent Chunks in {parent_retrieval_duration:.2f} seconds")

    # Step 4: Prepare contexts
    parent_context = "\n\n".join([doc.page_content for doc in parent_results])
    child_context = "\n\n".join([doc.page_content for doc in retrieved_children])

    # Step 5: Format the final prompt with query, retrieved context, and history summary
    formatted_prompt = serff_prompt_template.format(
        query=standalone_query,
        parent_context=parent_context,
        child_context=child_context,
        history_summary=history_summary
    )

    # Step 6: Generate final response from LLM
    start_time = time.time()
    response = llm.invoke(formatted_prompt, max_tokens=MAX_RESPONSE_TOKENS)
    response_duration = time.time() - start_time
    response_tokens = count_tokens(formatted_prompt) + count_tokens(response.content)
    answer = response.content.strip()
    print(f"💬 LLM Response Generated in {response_duration:.2f} seconds | Tokens Used: {response_tokens}")
    print(f"💬 Response: {answer}")

    # Step 7: Update conversation history with the reformulated query and response
    update_conversation_history(standalone_query, answer)

    return answer


In [22]:
get_conversation_context()
clear_history()

🧹 Conversation history cleared!


In [10]:
def on_state_change(selected_state):
    global hybrid_retriever
    hybrid_retriever = initialize_retriever(selected_state)


In [11]:
# ✅ Step 1: User selects a state
selected_state = None # Example from UI dropdown
hybrid_retriever = initialize_retriever(selected_state=selected_state)

🔎 No filter applied. Retrieving from all documents.


In [12]:
# First query
query1 = "What is Binding Authority for Crop Insurance Underwriting for Indiana"
response1 = retrieve_and_answer_the_query(query1, llm, hybrid_retriever)



🔍 Searching for: What is Binding Authority for Crop Insurance Underwriting for Indiana
What is Binding Authority for Crop Insurance Underwriting for Indiana
✅ Retrieved 4 Child Chunks in 0.88 seconds
📥 Fetching Parent Chunks for Parent_IDs: ['indiana:7:0', 'indiana:1:0', 'indiana:2:0', 'general:15:2']
✅ Retrieved 4 Parent Chunks in 0.41 seconds
💬 LLM Response Generated in 2.09 seconds | Tokens Used: 2370
💬 Response: Based on the provided context, the answer to the user query "What is Binding Authority for Crop Insurance Underwriting for Indiana?" is:

- Coverage for AABIC private insurance policies is bound by submitting a properly completed, signed, and dated application.
- An application is submitted when it has been properly keyed and scanned into the processing system.
- Coverage can be bound in two ways:
  1. Policy provisions shall take effect at 12:01 a.m. the following day after the application is signed and dated by the policyholder and agent provided that the complete applica

In [24]:
# First query
query2 = "List Form Code and IPA factor for South Dakota"
response2  = retrieve_and_answer_the_query(query2, llm, hybrid_retriever)



🔍 Searching for: List Form Code and IPA factor for South Dakota
📝 History Summary Generated in 0.73 seconds | Tokens Used: 207
🔄 Reformulated Query: What are the Form Code and IPA factor for South Dakota? | Time: 0.21s | Tokens: 145
What are the Form Code and IPA factor for South Dakota?
✅ Retrieved 4 Child Chunks in 0.27 seconds
📥 Fetching Parent Chunks for Parent_IDs: ['south_dakota:2:6', 'south_dakota:1:6', 'general:20:1']
✅ Retrieved 3 Parent Chunks in 0.08 seconds
💬 LLM Response Generated in 1.68 seconds | Tokens Used: 2292
💬 Response: Based on the provided context, the Form Code and IPA factor for South Dakota are as follows:

- Form Code: Companion 2, Companion 3, Companion 70
- IPA Factor:
  - Companion 2: 1.50
  - Companion 3: 2.00
  - Companion 70: 2.66

It is important to note that these are the maximum IPA factors for insureds with 100% share and listed maximum IPA of $1,000. Any requests for coverage beyond the stated limits must receive approval by AABIC, and approval req

In [25]:
# First query
query3 = "what is the Catastrophe Loss Award Policy"
response3  = retrieve_and_answer_the_query(query3, llm, hybrid_retriever)



🔍 Searching for: what is the Catastrophe Loss Award Policy
📝 History Summary Generated in 0.96 seconds | Tokens Used: 461
🔄 Reformulated Query: Can you provide information on the Catastrophe Loss Award Policy? | Time: 0.23s | Tokens: 367
Can you provide information on the Catastrophe Loss Award Policy?
✅ Retrieved 5 Child Chunks in 0.33 seconds
📥 Fetching Parent Chunks for Parent_IDs: ['colorado:2:8', 'colorado:1:6', 'iowa:1:8', 'general:3:0', 'general:20:2']
✅ Retrieved 5 Parent Chunks in 0.14 seconds
💬 LLM Response Generated in 2.04 seconds | Tokens Used: 2863
💬 Response: Based on the provided document context, the following information can be provided regarding the Catastrophe Loss Award Policy:

- The Catastrophe Loss Award applies to all policies below that pay 70% indemnity at 70% loss.
- Once the ascertained percentage of loss per acre exceeds 70%, an additional one-half of the percent of loss that is in excess of 70% will be paid. However, the total percentage payable shall not

In [26]:
# First query
query4 = "Describe DDB, DDC, DDA definition for South Dakota "
response4  = retrieve_and_answer_the_query(query4, llm, hybrid_retriever)



🔍 Searching for: Describe DDB, DDC, DDA definition for South Dakota 
📝 History Summary Generated in 1.10 seconds | Tokens Used: 737
🔄 Reformulated Query: What are the definitions of DDB, DDC, and DDA in South Dakota? | Time: 0.25s | Tokens: 639
What are the definitions of DDB, DDC, and DDA in South Dakota?
✅ Retrieved 5 Child Chunks in 0.28 seconds
📥 Fetching Parent Chunks for Parent_IDs: ['south_dakota:2:0', 'south_dakota:2:10', 'south_dakota:1:19', 'south_dakota:2:23', 'south_dakota:1:21']
✅ Retrieved 5 Parent Chunks in 0.14 seconds
💬 LLM Response Generated in 2.67 seconds | Tokens Used: 4290
💬 Response: Based on the provided document context, the definitions of DDB, DDC, and DDA in South Dakota are as follows:

- DDA: Provides a loss payment when the adjusted loss per acre exceeds 10%. The percentage per acre then payable will be the percent in excess of 10%. When the percent of loss exceeds 20%, an additional 2% will be paid for each percent in excess of 20%. Once the adjusted loss

In [313]:
import pandas as pd

# Set Pandas options to display all rows and columns
pd.set_option('display.max_rows',5)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_colwidth', None)  # Show full content of each column

# Now display the DataFrame
pd.DataFrame(child_db.get(include=["metadatas", "documents"]))


,ids,embeddings,metadatas,documents,uris,data
0,00158af4-3417-4a40-bcd7-eccf842eb5b1,None,"{'ID': 'south_dakota:1:14:5', 'LOB': 'American Agri-Business Insurance Company', 'Parent_ID': 'south_dakota:14', 'State': 'south_dakota', 'chunk_size': 400, 'filename': 'ARMT-132998838-2022 SD Crop Hail Rules 12.08.21.pdf', 'related_to': 'south_dakota:1:14'}",2020 -NCIS 562 \nThis mandatory endorsement amends your policy for coverage on hemp grown for \nharvest as fiber. XS10 deductible. Premium for this endorsement is due at the time of \napplication. \n \nSOUTH DAKOTA OPTIONAL ENDORSEMENTS \nIntroduction Below are general descriptions of the optional endorsements available with a Crop Hail,None,None
1,0023319d-1207-4ee9-adce-ff0acc984cb4,None,"{'ID': 'indiana:3:14:5', 'LOB': 'American Agri-Business Insurance Company', 'Parent_ID': 'indiana:14', 'State': 'indiana', 'chunk_size': 400, 'filename': 'ARMT-132998817-2022 IN Rates (Formatted) 08.24.21.pdf', 'related_to': 'indiana:3:14'}",105 Monroe CP2 0.50 1.10 0.70 7.45 1.25 0.85\n105 Monroe CP3 0.70 1.50 1.00 9.95 1.75 1.20\n105 Monroe CP4 0.85 1.85 1.20 11.50 2.15 1.50\n105 Monroe XS5IP 4.10\n105 Monroe XS10IP 3.20\n105 Monroe XS15 9.25\n105 Monroe XS15IP 1.60 3.10 3.35\n105 Monroe XS20 0.80\n105 Monroe XS10 0.80\n107 Montgomery BASIC 0.45 1.00 0.65 2.15 4.15 4.50 1.15 0.80\n107 Montgomery DXS5 0.30 0.75 0.45 0.75 0.55,None,None
...,...,...,...,...,...,...
3779,ffd8de8b-b37f-44f3-9705-dd45fd6c1964,None,"{'ID': 'indiana:6:0:2', 'LOB': 'American Agri-Business Insurance Company', 'Parent_ID': 'indiana:0', 'State': 'indiana', 'chunk_size': 400, 'filename': 'ARMT-132998817-IN Objection Responses 01.26.22.pdf', 'related_to': 'indiana:6:0'}","Objection 2: \nCompany experience and/or competitive comparisons that led to the Wind Plus/Extra Harvest Expense \nwind rate adjustments, along with the actual adjustments. \n \nResponse 2: \nSee the graphs below and on the next page for competitive comparisons and company adjustments .",None,None
3780,ffe171bd-172e-4bc8-99eb-1248c1031c95,None,"{'ID': 'iowa:4:184:1', 'LOB': 'American Agri-Business Insurance Company', 'Parent_ID': 'iowa:184', 'State': 'iowa', 'chunk_size': 400, 'filename': 'ARMT-132998818-2022 IA Production Wind Rates (Formatted) 12.03.21.pdf', 'related_to': 'iowa:4:184'}",191 Winneshiek 105/50 3.10 3.00 3.65 191 Winneshiek 105/50 3.55 3.40 4.15\n191 Winneshiek 105/55 3.40 3.25 3.95 191 Winneshiek 105/55 3.85 3.70 4.50\n191 Winneshiek 105/60 3.70 3.55 4.35 191 Winneshiek 105/60 4.25 4.05 4.95\n191 Winneshiek 105/65 4.10 3.95 4.80 191 Winneshiek 105/65 4.70 4.50 5.45\n191 Winneshiek 105/70 4.55 4.40 5.30 191 Winneshiek 105/70 5.20 5.00 6.05,None,None


In [68]:


    def reformulate_question(self, query_text, llm):
        """
        Reformulate the follow-up question using LLM based on the conversation history.
        """
        # Get the conversation history
        conversation_context = self.get_conversation_context()

        # Use the reformulation prompt to generate a standalone question
        reformulation_prompt = """
        Given a chat history and the latest user question which might reference context in the chat history, 
        formulate a standalone question which can be understood without the chat history. Do NOT answer the question, 
        just reformulate it if needed and otherwise return it as is.

        Chat History:
        {chat_history}

        Latest Question:
        {latest_question}
        """

        # Prepare the input for the LLM model
        formatted_prompt = reformulation_prompt.format(chat_history=conversation_context, latest_question=query_text)

        # Get the reformulated question from LLM
        reformulated_response = llm.invoke(input=formatted_prompt)
        standalone_question = reformulated_response.content.strip()

        return standalone_question


### making a chain of conversation

In [16]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import AIMessage, HumanMessage

# Assuming `retriever` is your document retrieval mechanism, already defined
db = Chroma(
        persist_directory=CHROMA_PATH, embedding_function=get_embedding()
    )
retriever = db.as_retriever(search_type="mmr",
    search_kwargs={'k': 20,'lambda_mult': 0.05}
)  # Define your document retriever here

# Contextualization prompt template
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# Create the history-aware retriever chain
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [ ]:
retriever